# Import Modules

In [28]:
%reload_ext autoreload
%autoreload 2
from PdfReader import pdf_to_docVec, pdf_to_text
from DocVec import DocVec
from Qdrant import Qdrant

# Encoding documents and paragraphs

In [1]:
from sentence_transformers import SentenceTransformer
encoder = SentenceTransformer("all-MiniLM-L6-v2")

c:\Program Files\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [106]:
from os import listdir
from os.path import isfile, join
dir_path = "D:\\Master1\\Software\\testDocs"
all_docs = [join(dir_path, f) for f in listdir(dir_path) if isfile(join(dir_path, f))]
all_docs

['D:\\Master1\\Software\\testDocs\\Darth Revan.pdf',
 'D:\\Master1\\Software\\testDocs\\Forschung_22.pdf',
 'D:\\Master1\\Software\\testDocs\\Praktikum.pdf',
 'D:\\Master1\\Software\\testDocs\\voegel.pdf',
 'D:\\Master1\\Software\\testDocs\\ZulassungVorrausetzung_master.pdf']

In [107]:
documents = [pdf_to_docVec(doc, encoder) for doc in all_docs]

# Creating a qdrant client

In [29]:
from qdrant_client import QdrantClient
client = Qdrant(encoder=encoder,
                qdrant_client=QdrantClient("localhost", port=6333))

In [14]:
# method to test the search result
def test_search(qa, username):
      for q in qa:
            ans = client.search(username, q['question'])['relevant_docs']
            act = q['answer']
            print(f"answer is {ans}, should be {act}")

# Adding vectors
- A collection for the user name will be automatically created, if it doesn't exist.

- Otherwise the vectors will be added to the existing collection

In [ ]:
userName = "testUser"

for document in documents:
  client.add_docVec(userName, document)

user doesn't exist, creating a new collection with the user name: testUser
Darth Revan, eine der faszinierendsten Figuren im Star Wars -Universum, ist eine zentrale Figur in der 
Ära der Alten Republik und in der Handlung vieler Star Wars -Videospiele und Romane. Seine 
Geschichte ist geprägt von einer tiefgreifenden Wandlung von ein em gefeierten Jedi -Ritter zu einem 
mächtigen Sith -Lord und später zu einem Helden, der seine dunkle Vergangenheit überwindet, um 
erneut das Gute zu verteidigen.  
 
Die Geschichte von Darth Revan beginnt in den Zeiten der Alten Republik, einer Ära des Friedens und 
des Wohlstands, in der die Jedi -Ritter als Hüter des Friedens und der Gerechtigkeit bekannt sind
Revan, ursprünglich als ein hervorragender Jedi bekannt, w ar eine talentierte und charismatische 
Figur, die für seine Weisheit und seine Fähigkeiten in der Macht bewundert wurde. Gemeinsam mit 
seinem treuen Freund Malak führte er die Republik im Krieg gegen die Sith und galt als ein 
leuchten

# Delete all vectors

In [5]:
import os
import sys
sys.path.append("../")
import config

def create_directory_structure_list(root_directory):
    structure_list = []
    for root, dirs, files in os.walk(root_directory):
        if len(dirs) == 0:

            current_dict = {'username': os.path.basename(root), 'files': [os.path.join(root, file) for file in files]}
            structure_list.append(current_dict)
    return structure_list

directory_path = config.document_directory
users_files = create_directory_structure_list(directory_path)

print(users_files)

[{'username': 'test1', 'files': ['D:\\Master1\\Software\\git\\mindmachine\\backend\\root\\mm_docs\\user\\test1\\Forschung_22.pdf', 'D:\\Master1\\Software\\git\\mindmachine\\backend\\root\\mm_docs\\user\\test1\\Praktikum.pdf']}, {'username': 'test2', 'files': ['D:\\Master1\\Software\\git\\mindmachine\\backend\\root\\mm_docs\\user\\test2\\Darth Revan.pdf', 'D:\\Master1\\Software\\git\\mindmachine\\backend\\root\\mm_docs\\user\\test2\\voegel.pdf']}]


In [10]:
client.revectorize_all()

deleting test1
D:\Master1\Software\git\mindmachine\backend\root\mm_docs\user\test1\Forschung_22.pdf
user doesn't exist, creating a new collection with the user name: test1
Die Forschung an deutschen Universitäten nimmt eine herausragende Stellung in der internationalen Wissenschaftslandschaft ein Deutschland ist bekannt für seine exzellente Forschung und seine zahlreichen renommierten Universitäten die eine breite Palette von Disziplinen abdecken Hier sind einige wichtige Aspekte der Forschung an deutschen Universitäten   Forschungslandschaft  Deutschland verfügt über eine vielfältige Forschungslandschaft mit einer beeindruckenden Bandbreite an Forschungsbereichen und Disziplinen Dies umfasst Naturwissenschaften Ingenieurwissenschaften Geistes  und Sozialwissenschaften Medizin und viele weit ere Fachgebiete 
 Forschungsförderung  Die deutsche Regierung und verschiedene Organisationen investieren erhebliche Mittel in die Forschung Das Bundesministerium für Bildung und Forschung BMBF sow

# rename vectors

In [23]:
from qdrant_client import models
def rename_doc(collection_name, doc_name, new_name):
    rename_vec(collection_name, doc_name, new_name, "name")
    rename_vec(collection_name, doc_name, new_name, "source_doc")

def rename_vec(collection_name, doc_name, new_name, key):
    client.qdrant_client.set_payload(
    collection_name=collection_name,
    payload={
        key: new_name,
    },
    points=models.Filter(
      must=[
          models.FieldCondition(
              key=key,
              match=models.MatchValue(value=doc_name),
          ),
      ],
    ),
    )
    return True

In [31]:
client.rename_doc("test1","Forschung_22.pdf_renamed","Praktikum.pdf")

True

# preprocessing

## before special chars excluded vs after
- testUser: Special chars are not excluded
- UserPre: Special chars are excluded
* Result: no difference. But the special chars are anyways not relevent to the context, they better be removed 

In [96]:
QA_raw = [{'question':"Was sind die Zulassungsvoraussetzungen für Ingenuirinformatik", 'answer':"ZulassungVorrausetzung_master.pdf"},
      {'question':"Wer investiert in der Forschung in Deutschland", 'answer':"Forschung_22.pdf"},
      {'question':"Was sind die wichtige Schritte bei der Bewerbung um ein Praktikum an der HTW", 'answer':"Praktikum.pdf"},
      {'question':"Welche Unterlagen sind für Praktikum Bewerbung an der HTW erforderlich", 'answer':"Praktikum.pdf"},
      {'question':"Welche Rolle spielen Vögel in vielen Ökosystemen?", 'answer':"voegel.pdf"},
      {'question':"Welche Herausforderungen sind viele Vogelarten heutzutage ausgesetzt?", 'answer':"voegel.pdf"}]

In [15]:
test_search(QA_raw, "testUser")

{'isDoc': True, 'name': 'D:\\Master1\\Software\\testDocs\\Praktikum.pdf'} score: 0.28322232
{'isDoc': True, 'name': 'D:\\Master1\\Software\\testDocs\\Forschung_22.pdf'} score: 0.27045977
{'isDoc': True, 'name': 'D:\\Master1\\Software\\testDocs\\ZulassungVorrausetzung_master.pdf'} score: 0.25434506
{'isDoc': True, 'name': 'D:\\Master1\\Software\\testDocs\\Darth Revan.pdf'} score: 0.16957936
{'isDoc': True, 'name': 'D:\\Master1\\Software\\testDocs\\voegel.pdf'} score: 0.16722113


NameError: name 'relevant_doc' is not defined

In [22]:
len(client.search("UserPre", "zulassung")['relevant_paragraph'])

{'isDoc': True, 'name': 'D:\\Master1\\Software\\testDocs\\ZulassungVorrausetzung_master.pdf'} score: 0.2684761
{'isDoc': True, 'name': 'D:\\Master1\\Software\\testDocs\\Praktikum.pdf'} score: 0.22633848
{'isDoc': True, 'name': 'D:\\Master1\\Software\\testDocs\\Forschung_22.pdf'} score: 0.2012557
{'isDoc': True, 'name': 'D:\\Master1\\Software\\testDocs\\voegel.pdf'} score: 0.13682225
{'isDoc': True, 'name': 'D:\\Master1\\Software\\testDocs\\Darth Revan.pdf'} score: 0.037946925
{'isDoc': False, 'name': '2 Erfüllt ein oder eine Bewerberin mehrere der angegebenen Kriterien so wird dasjenige mit der besten Note berücksichtigt Wird gar kein Kriterium erfüllt so erfolgt eine Bewertung mit der Note 40 im Zulassungsverfahren', 'source_doc': 'D:\\Master1\\Software\\testDocs\\ZulassungVorrausetzung_master.pdf'} score: 0.30355325
{'isDoc': False, 'name': '   5 Frist und Form der Bewerbung 1 Frist und Form der Bewerbung regelt die Auswahlordnung für konsekutive Masterstudiengänge der HTW Berlin AOM

8

In [98]:
test_search(QA_raw, "UserPre")

0.28322232
0.2982448
answer is D:\Master1\Software\testDocs\Praktikum.pdf, should be ZulassungVorrausetzung_master.pdf
0.47206515
0.61208105
answer is D:\Master1\Software\testDocs\Forschung_22.pdf, should be Forschung_22.pdf
0.6100899
0.60854244
answer is D:\Master1\Software\testDocs\Praktikum.pdf, should be Praktikum.pdf
0.3966717
0.4148323
answer is D:\Master1\Software\testDocs\Praktikum.pdf, should be Praktikum.pdf
0.535673
0.5353848
answer is D:\Master1\Software\testDocs\voegel.pdf, should be voegel.pdf
0.60158694
0.5710124
answer is D:\Master1\Software\testDocs\voegel.pdf, should be voegel.pdf


## Removing stop words

### removing question word from the query

In [99]:
QA_no_question_word = [{'question':"sind die Zulassungsvoraussetzungen für Ingenuirinformatik", 'answer':"ZulassungVorrausetzung_master.pdf"},
                        {'question':"investiert Forschung Deutschland", 'answer':"Forschung_22.pdf"},
                        {'question':"sind di wichtige Schritte bei der Bewerbung um ein Praktikum an der HTW", 'answer':"Praktikum.pdf"},
                        {'question':"Unterlagen sind für Praktikum Bewerbung an der HTW erforderlich", 'answer':"Praktikum.pdf"},
                        {'question':"Rolle spielen Vögel in vielen Ökosystemen", 'answer':"voegel.pdf"},
                        {'question':"Herausforderungen sind viele Vogelarten heutzutage ausgesetzt", 'answer':"voegel.pdf"}]

In [100]:
test_search(QA_no_question_word, "UserPre")

0.2728484
0.29592583
answer is D:\Master1\Software\testDocs\Praktikum.pdf, should be ZulassungVorrausetzung_master.pdf
0.4568019
0.56380147
answer is D:\Master1\Software\testDocs\Forschung_22.pdf, should be Forschung_22.pdf
0.59114444
0.59752107
answer is D:\Master1\Software\testDocs\Praktikum.pdf, should be Praktikum.pdf
0.34880492
0.36860758
answer is D:\Master1\Software\testDocs\Praktikum.pdf, should be Praktikum.pdf
0.5078301
0.4995128
answer is D:\Master1\Software\testDocs\voegel.pdf, should be voegel.pdf
0.54649305
0.53457
answer is D:\Master1\Software\testDocs\voegel.pdf, should be voegel.pdf


### removing (Artikel und Präpositionen)

In [110]:
QA_no_artikel = [{'question':"sind Zulassungsvoraussetzungen Ingenuirinformatik", 'answer':"ZulassungVorrausetzung_master.pdf"},
                        {'question':"investiert Forschung Deutschland", 'answer':"Forschung_22.pdf"},
                        {'question':"sind wichtige Schritte Bewerbung Praktikum HTW", 'answer':"Praktikum.pdf"},
                        {'question':"Unterlagen sind Praktikum Bewerbung HTW erforderlich", 'answer':"Praktikum.pdf"},
                        {'question':"Rolle spielen Vögel vielen Ökosystemen", 'answer':"voegel.pdf"},
                        {'question':"Herausforderungen sind viele Vogelarten heutzutage ausgesetzt", 'answer':"voegel.pdf"}]

In [111]:
test_search(QA_no_artikel, "UserPre")

0.26040825
0.29681665
answer is D:\Master1\Software\testDocs\Praktikum.pdf, should be ZulassungVorrausetzung_master.pdf
0.4568019
0.56380147
answer is D:\Master1\Software\testDocs\Forschung_22.pdf, should be Forschung_22.pdf
0.5077378
0.51655805
answer is D:\Master1\Software\testDocs\Praktikum.pdf, should be Praktikum.pdf
0.29118145
0.32828966
answer is D:\Master1\Software\testDocs\Praktikum.pdf, should be Praktikum.pdf
0.46307644
0.45769283
answer is D:\Master1\Software\testDocs\voegel.pdf, should be voegel.pdf
0.54649305
0.53457
answer is D:\Master1\Software\testDocs\voegel.pdf, should be voegel.pdf


### Removing (sein)

In [112]:
QA_no_sein = [{'question':"Zulassungsvoraussetzungen Ingenuirinformatik", 'answer':"ZulassungVorrausetzung_master.pdf"},
                        {'question':"investiert Forschung Deutschland", 'answer':"Forschung_22.pdf"},
                        {'question':"wichtige Schritte Bewerbung Praktikum HTW", 'answer':"Praktikum.pdf"},
                        {'question':"Unterlagen Praktikum Bewerbung HTW erforderlich", 'answer':"Praktikum.pdf"},
                        {'question':"Rolle spielen Vögel vielen Ökosystemen", 'answer':"voegel.pdf"},
                        {'question':"Herausforderungen viele Vogelarten heutzutage ausgesetzt", 'answer':"voegel.pdf"}]

In [113]:
test_search(QA_no_sein, "UserPre")

0.2559325
0.42462927
answer is D:\Master1\Software\testDocs\ZulassungVorrausetzung_master.pdf, should be ZulassungVorrausetzung_master.pdf
0.4568019
0.56380147
answer is D:\Master1\Software\testDocs\Forschung_22.pdf, should be Forschung_22.pdf
0.51693237
0.53008485
answer is D:\Master1\Software\testDocs\Praktikum.pdf, should be Praktikum.pdf
0.31886622
0.3616893
answer is D:\Master1\Software\testDocs\Praktikum.pdf, should be Praktikum.pdf
0.46307644
0.45769283
answer is D:\Master1\Software\testDocs\voegel.pdf, should be voegel.pdf
0.5276303
0.5208548
answer is D:\Master1\Software\testDocs\voegel.pdf, should be voegel.pdf


# Pargaraphs

* handling tables:
    - tabula-py: requiers java
    - pymupdf: requiers c/c++ devTools

In [9]:
doc = pdf_to_text("D:\\Master1\\Software\\testDocs\\ZulassungVorrausetzung_master.pdf")

In [11]:
doc['text']

'Nr. 01/23 Amtliches Mitteilungsblatt der HTW Berlin Seite 1 \n \n  10. März 2023 \nAmtliches Mitteilungsblatt \n \nSeite\nZugangs- und Zulassungsordnung für den konsekutiven \nMasterstudiengang Informatik in Ingenieurwissenschaften \nim Fachbereich Ingenieurwissenschaften – Technik und  \nLeben  vom 13. Oktober 2021 . . . . . . . . . . . . . . . . . . . . . . . . .    3  \n \n \n \nSeite 2 Amtliches Mitteilungsblatt der HTW Berlin Nr. 01/23  \n  \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n  \n \n \n \n \n \n \n \nHerausgeberin \nDie Hochschulleitung der HTW Berlin \nTreskowallee 8 \n10318 Berlin \n \nRedaktion \nJustiziariat \nTel. +49 30 5019-2813 \nFax +49 30 5019-2815 Nr. 01/23 Amtliches Mitteilungsblatt der HTW Berlin Seite 3 \n \nHochschule für Technik und Wirtschaft Berlin \n \nZugangs- und Zulassungsordnung \nfür den konsekutiven Masterstudiengang \n \nInformatik in Ingenieurwissenschaften (IIW) \n \nim Fachbereich Ingenieurwissen

In [15]:

vec = encoder.encode(doc['text']).tolist()
vecs = encoder.encode(doc['paragraphs']).tolist()

paras_vecs = []
for idp, (para, vec) in enumerate(zip(doc['paragraphs'], vecs)):
    paras_vecs.append({"paragraph": para, "vec": vec})

In [16]:
paras_vecs[1]

{'paragraph': '1039 und von  17 Abs 1 Nr 1 Neufassung der Satzung der Hochschule für Technik und Wirtschaft Berlin zu Abweichungen von Bestimmungen des Berliner Hochschulgesetzes AMBl HTW Berlin Nr 2909 in Verbindung mit  10 Abs 5 Satz 1 des Gesetzes über die Hochschulen im Land Berlin Berliner Hochschulgesetz  BerlHG in der Fassung der Bekanntmachung vom 26 Juli 2011 GVBl S',
 'vec': [-0.01948305405676365,
  -0.004690532572567463,
  -0.005532553885132074,
  -0.048161499202251434,
  -0.01918713003396988,
  0.052032992243766785,
  -0.0038676923140883446,
  0.09741304069757462,
  -0.14600268006324768,
  0.003540483070537448,
  -0.05239804461598396,
  -0.029526913538575172,
  -0.0001182716732728295,
  -0.03926537185907364,
  -0.05346233770251274,
  -0.09782367944717407,
  -0.034825146198272705,
  -0.07363630086183548,
  -0.017508111894130707,
  -0.1477915197610855,
  0.002671771449968219,
  0.01997099071741104,
  -0.0031421189196407795,
  -0.050201691687107086,
  0.07973342388868332,
  -0

# Tokenization
Research output:
* Does it make difference to send the scentences seperetly or all at once to the model?
    - yes, the attention mask will be differnt. But we don't need this for our usecase.
        - If we tokenize all the scentences together we will get some zeros in the attention mask, the reason of this is that shorter sentences get more tokens than they need and the token that crossponds with a zero should be ignored.
* Does this difference affect our usecase?
    - No, as long as the input_id of each spesific token is always the same, this will not affect our vector representation    

In [32]:
from transformers import AutoTokenizer
import torch
import torch.nn.functional as F

In [33]:
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0] 
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)

In [34]:
def tokenize(sentences):
    tokenizer = AutoTokenizer.from_pretrained('sentence-transformers/all-MiniLM-L6-v2')
    return tokenizer(sentences, padding=True, truncation=True, return_tensors='pt')

In [35]:
def encode_tokens(tokens, model):
    with torch.no_grad():
        model_output = model(**tokens)

    sentence_embeddings = mean_pooling(model_output, tokens['attention_mask'])

    sentence_embeddings = F.normalize(sentence_embeddings, p=2, dim=1)
    return sentence_embeddings.tolist()

In [49]:
long_sentence = ["Forschung hat viele vorteile die man nützen kann um die Lebensqualität zu verbessern Neue Erkenntnisse können dazu beitragen effektivere Medikamente zu entwickeln nachhaltige Energiequellen zu finden und innovative Technologien zu schaffen"]
encoded_input_one_scentence = tokenize(long_sentence)
encoded_input_one_scentence['input_ids'].shape, encoded_input_one_scentence['attention_mask']

(torch.Size([1, 79]),
 tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
          1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
          1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
          1, 1, 1, 1, 1, 1, 1]]))

In [53]:
short_sentence = ["Informatik ist die Wahl studiengang"]
encoded_input_one_scentence = tokenize(short_sentence)
encoded_input_one_scentence['input_ids'], encoded_input_one_scentence['attention_mask'], encoded_input_one_scentence['token_type_ids']

(tensor([[  101, 12367, 10450,  2243, 21541,  3280, 22894,  2140, 16054,  9013,
          24930,   102]]),
 tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]),
 tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]))

In [63]:
test_sentences = ["Informatik ist die Wahl studiengang",
                  "was ist die beste Option an der fachschule?",
                  "Forschung hat viele vorteile die man nützen kann um die Lebensqualität zu verbessern Neue Erkenntnisse können dazu beitragen effektivere Medikamente zu entwickeln nachhaltige Energiequellen zu finden und innovative Technologien zu schaffen"]
encoded_input_three_scentence = tokenize(test_sentences)

In [64]:
# the shape of the same scentence is different, when encoded with another scentence
encoded_input_three_scentence['input_ids'][0].shape, encoded_input_three_scentence['input_ids'][1].shape

(torch.Size([79]), torch.Size([79]))

In [65]:
# but the content of the input_ids is still the same. All the additional 0 are acting like padding items, to produce the same shape of the longer sentence. And they will be ignored anyways with the crossponding zero in the attention mask. 
encoded_input_three_scentence['input_ids'][0]

tensor([  101, 12367, 10450,  2243, 21541,  3280, 22894,  2140, 16054,  9013,
        24930,   102,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0])

In [66]:
# token_type_ids: Are not relevent in the case of all-MiniLM-L6-v2. They will be always zero
encoded_input_three_scentence['token_type_ids'][0], encoded_input_three_scentence['token_type_ids'][1], encoded_input_three_scentence['token_type_ids'][1] 

(tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0]),
 tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0]),
 tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0]))

## try to encode all the documents at the same time

In [70]:
"""
PDF Helper Functions

Author: Abdelrahman Elsharkawi
Creation Date: 11.11.2023
"""
import PyPDF2
from Helper_Modules.DocVec import DocVec
import re

def exclude_special_characters(input_string):
    """
    Exclude all chars except numbers and letters from the input string.

    Parameters:
    - input_string (str): Input string.

    Returns:
    str: String with special characters removed.
    """
    result = re.sub(r'[^\w\s]', '', input_string.replace('\n', ''))
    return result

def pdf_to_text(path):
  """
  Convert PDF file to plain text.

  Parameters:
  - path (str): Path to the PDF file.

  Returns:
  dict: {'text':extracted_text, 'paragraphs':paragraphs}.
  """
  pdffileobj=open(path,'rb')
  pdfreader=PyPDF2.PdfReader(pdffileobj)
  extracted_text = ""

  for page in pdfreader.pages:
    extracted_text += page.extract_text()
    paragraphs = [exclude_special_characters(p) for p in extracted_text.split(". \n")]

  return {'text':extracted_text, 'paragraphs':paragraphs}

def pdf_to_docVec(paths, encoder):
  """
  Convert PDF file to DocVec object.

  Parameters:
  - path (str): Path to the PDF file.
  - encoder: Sentence embeddings encoder.

  Returns:
  DocVec: DocVec object containing document vectors and paragraph vectors.
  """
  docs_text = []
  docs_complete = []
  docs_vecs = []
  for path in paths:
    text_paragraphs = pdf_to_text(path)
    docs_complete.append(text_paragraphs)
    docs_text.append(text_paragraphs['text'])
  print(len(docs_complete))
  texts_vecs = encoder.encode(docs_text).tolist()
  print(len(texts_vecs))
  print(len(texts_vecs[0]))

  for idp,doc in enumerate(texts_vecs):
    print(idp)
    vecs = encoder.encode(docs_complete[idp]['paragraphs']).tolist()

    paras_vecs = []
    for idx, (para, vec) in enumerate(zip(docs_complete[idp]['paragraphs'], vecs)):
        paras_vecs.append({"paragraph": para, "vec": vec})
    docs_vecs.append(DocVec(paths[idp], doc, paras_vecs))
    

  

  return docs_vecs

In [71]:
documents = [pdf_to_docVec(all_docs, encoder)]

5
5
384
0
1
2
3
4


In [79]:
[print(i.paras_vecs) for i in documents[0]]

[{'paragraph': 'Darth Revan eine der faszinierendsten Figuren im Star Wars Universum ist eine zentrale Figur in der Ära der Alten Republik und in der Handlung vieler Star Wars Videospiele und Romane Seine Geschichte ist geprägt von einer tiefgreifenden Wandlung von ein em gefeierten Jedi Ritter zu einem mächtigen Sith Lord und später zu einem Helden der seine dunkle Vergangenheit überwindet um erneut das Gute zu verteidigen   Die Geschichte von Darth Revan beginnt in den Zeiten der Alten Republik einer Ära des Friedens und des Wohlstands in der die Jedi Ritter als Hüter des Friedens und der Gerechtigkeit bekannt sind', 'vec': [-0.04045562073588371, 0.04286428913474083, -0.020568300038576126, -0.08114031702280045, -0.06727588176727295, 0.020186608657240868, -0.008466239087283611, -0.0060557592660188675, 0.06957694888114929, 0.02513938583433628, 0.035703904926776886, 0.05240420252084732, -0.01000794768333435, -0.062104806303977966, -0.06947408616542816, -0.07673589885234833, -0.058148182

[None, None, None, None, None]

In [82]:
userName = "DocsEncodedTogether"

for document in documents[0]:
  client.add_docVec(userName, document)

user: DocsEncodedTogether exists, and has 0 vectors
Darth Revan eine der faszinierendsten Figuren im Star Wars Universum ist eine zentrale Figur in der Ära der Alten Republik und in der Handlung vieler Star Wars Videospiele und Romane Seine Geschichte ist geprägt von einer tiefgreifenden Wandlung von ein em gefeierten Jedi Ritter zu einem mächtigen Sith Lord und später zu einem Helden der seine dunkle Vergangenheit überwindet um erneut das Gute zu verteidigen   Die Geschichte von Darth Revan beginnt in den Zeiten der Alten Republik einer Ära des Friedens und des Wohlstands in der die Jedi Ritter als Hüter des Friedens und der Gerechtigkeit bekannt sind
Revan ursprünglich als ein hervorragender Jedi bekannt w ar eine talentierte und charismatische Figur die für seine Weisheit und seine Fähigkeiten in der Macht bewundert wurde Gemeinsam mit seinem treuen Freund Malak führte er die Republik im Krieg gegen die Sith und galt als ein leuchtendes Vorbild für den Je diOrden   Jedoch änderte si

# More structured tests

In [23]:
from sklearn.metrics import confusion_matrix
# TODO: impelement a confusion matrix 
def draw_confusion(qa, pred_answers):
    lables = list(set(d['answer'] for d in qa))
    actual = [d['answer'] for d in qa]
    preds = [d['relevant_doc'] for d in pred_answers]
    confusion_matrix(actual, preds, lables)

# trying BERT
Just exploring the model.
- we might use it to find the answer in the most relevent paragraph, or git rid of the most relevant paragraph thing and find the answer directly in the document
- still needs research 
    - How many words can i give as an answer?
    - Find a pretrained model which works well with german and english.

In [1]:
import torch
from transformers import BertForQuestionAnswering
from transformers import BertTokenizer
import warnings
warnings.simplefilter("ignore")

weight_path = "kaporter/bert-base-uncased-finetuned-squad"
tokenizer = BertTokenizer.from_pretrained(weight_path)
model = BertForQuestionAnswering.from_pretrained(weight_path)

c:\Program Files\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
question = "What are the usages of robotics in our dailz life?"
context = "Navigating the Future: The Rise of AI and Robotics Introduction In the ever-evolving landscape of technology, artificial intelligence (AI) and robotics stand at the forefront of innovation, driving changes that are reshaping our world. This article delves into the latest advancements in these fields, exploring how they are transforming industries, enhancing our daily lives, and challenging our understanding of what's possible. Section 1: AI Revolution AI technology has made leaps and bounds in recent years. From sophisticated machine learning algorithms to advancements in neural networks, AI is not just a futuristic concept but a present reality. We examine key breakthroughs in AI, including its application in healthcare for disease prediction, in finance for risk analysis, and in everyday gadgets that make life more convenient. Section 2: Robotics and Automation The world of robotics has seen remarkable growth, with robots becoming more autonomous, efficient, and integrated into human environments. We explore the latest in robotic technology, from industrial robots revolutionizing manufacturing to service robots in retail and healthcare. The implications of automation in the workforce and the ethical considerations it raises are also discussed. Section 3: AI and Robotics in Everyday Life AI and robotics are no longer confined to research labs and industries; they are part of our daily lives. From smart home devices to AI-driven personal assistants, these technologies are making life easier and more connected. We look at how AI and robotics are enhancing consumer experiences and changing the way we interact with technology.Section 4: Challenges and Future Directions Despite the exciting advancements, AI"

input_ids = tokenizer.encode(question, context)
print (f'We have about {len(input_ids)} tokens generated')

tokens = tokenizer.convert_ids_to_tokens(input_ids)
print(" ")
print('Some examples of token-input_id pairs:')

for i, (token,inp_id) in enumerate(zip(tokens,input_ids)):
    print(token,":",inp_id)

sep_idx = tokens.index('[SEP]')

token_type_ids = [0 for i in range(sep_idx+1)] + [1 for i in range(sep_idx+1,len(tokens))]
print(token_type_ids)
out = model(torch.tensor([input_ids]), 
                token_type_ids=torch.tensor([token_type_ids]))

start_logits,end_logits = out['start_logits'],out['end_logits']

answer_start = torch.argmax(start_logits)
answer_end = torch.argmax(end_logits)

ans = ''.join(tokens[answer_start:answer_end])
print('Predicted answer:', ans)

We have about 324 tokens generated
 
Some examples of token-input_id pairs:
[CLS] : 101
what : 2054
are : 2024
the : 1996
usage : 8192
##s : 2015
of : 1997
robotics : 21331
in : 1999
our : 2256
dail : 26181
##z : 2480
life : 2166
? : 1029
[SEP] : 102
na : 6583
##vi : 5737
##gating : 16961
the : 1996
future : 2925
: : 1024
the : 1996
rise : 4125
of : 1997
ai : 9932
and : 1998
robotics : 21331
introduction : 4955
in : 1999
the : 1996
ever : 2412
- : 1011
evolving : 20607
landscape : 5957
of : 1997
technology : 2974
, : 1010
artificial : 7976
intelligence : 4454
( : 1006
ai : 9932
) : 1007
and : 1998
robotics : 21331
stand : 3233
at : 2012
the : 1996
forefront : 22870
of : 1997
innovation : 8144
, : 1010
driving : 4439
changes : 3431
that : 2008
are : 2024
res : 24501
##ha : 3270
##ping : 4691
our : 2256
world : 2088
. : 1012
this : 2023
article : 3720
del : 3972
##ves : 6961
into : 2046
the : 1996
latest : 6745
advancement : 12607
##s : 2015
in : 1999
these : 2122
fields : 4249
, : 1010


[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


In [4]:
out = model(torch.tensor([input_ids]), 
                token_type_ids=torch.tensor([token_type_ids]))

start_logits,end_logits = out['start_logits'],out['end_logits']

answer_start = torch.argmax(start_logits)
answer_end = torch.argmax(end_logits)

ans = ''.join(tokens[answer_start:answer_end])
print('Predicted answer:', ans)

Predicted answer: visittheofficialwebsiteoftheh##t##wtogatherinformationaboutavailableinternshippositionsandtheapplication
